In [ ]:
# Function to compute color histogram in HSV space
def compute_color_histogram(image, bins=(8, 2, 2)):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_image], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Function to compute color moments (mean, std deviation)
def compute_color_moments(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mean, std = [], []
    for channel in cv2.split(hsv_image):
        mean.append(np.mean(channel))
        std.append(np.std(channel))
    return mean + std

# Function to compute Wavelet Transform
def compute_wavelet_features(image, wavelet='db1', level=3):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    coeffs = pywt.wavedec2(gray_image, wavelet, level=level)
    # Use mean and standard deviation of coefficients at each level
    features = []
    for coeff in coeffs:
        if isinstance(coeff, tuple):  # for details coefficients (horizontal, vertical, diagonal)
            for subband in coeff:
                features.append(np.mean(subband))
                features.append(np.std(subband))
        else:  # for the approximation coefficients
            features.append(np.mean(coeff))
            features.append(np.std(coeff))
    return features

# Function to compute LBP
def compute_lbp_features(image, radius=2, n_points=16):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_image, n_points, radius, method="uniform")
    # Create histogram of LBP
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    # Normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

# Combination of Handcrafted features for various experiments has been done by concatenate instructions

# Load a pre-trained EfficientNet-B7 model
model = models.efficientnet_b7(pretrained=True)
# Remove the final classification layer to get features from the penultimate layer
model = nn.Sequential(*list(model.children())[:-1])
model.eval()

# Transformation to resize images to 600x600 and normalize them (EfficientNet-B7 recommended input size)
transform = transforms.Compose([
    transforms.Resize((600, 600)),  # EfficientNet B7 recommended input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

